In [15]:
import time
import random
import datetime
import schedule
# dc_crawling.py 같은 폴더에 있다고 가정
#from dc_crawling import *from kafka impt KafkaProducer

In [16]:
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import time
import sqlite3
from requests import Session
from bs4 import BeautifulSoup
import re
import datetime
import pandas# as pd
import datetime
from tqdm import tqdm
from collections import defaultdict

In [17]:
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params=params if method.upper() == 'GET' else '',
                               data=params if method.upper() == 'POST' else '',
                               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60)  # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit - 1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)

    return resp

# 기존

In [6]:
#bootstrap_servers = ['localhost:9092','localhost:9093']  # kafka broker ip
bootstrap_servers = ['localhost:9092']
topicName = 'realtime_stock'
producer = KafkaProducer(bootstrap_servers = bootstrap_servers)

def stockprice():

    codes = {'336260', '009830', '086520', '011930', '018000', '035720', '035420', '036570', 
    '041140', '018260', '005930', '000660', '011790', '014680', '000990', '023530', '004170', 
    '008770', '028260', '026960', '105560', '071050', '055550', '316140', '086790', '047080', 
    '089890', '030350', '230980', '069410'}

    now = datetime.datetime.now()

    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

    # ticker 처음부터 다 돌기
    for code in codes:
        time = datetime.datetime(now.year, now.month, now.day)

        if now.weekday() == 5 or now.weekday() == 6:
            break

        result = list()
        pages = range(1, 13)
        for page in pages:
            url = 'https://finance.naver.com/item/sise_time.nhn?code={}&thistime={}&page={}'.format(code, time.strftime("%Y%m%d")+"161122", page)
            resp = download(url, headers = headers)
            dom = BeautifulSoup(resp.content, 'lxml')

            # result = for t, p in zip([i.text for i in dom.select('tr > td:nth-of-type(1) > span')], [i.text for i in dom.select('tr > td:nth-of-type(2) > span')])}
            for i in range(len(dom.select('tr > td:nth-of-type(2) > span'))):
                # result list에 dictionary 형태로 추가
                result.append({
                    
                    'code':code, 
                    'date': time.strftime("%Y.%m.%d") + " " + dom.select('tr > td:nth-of-type(1) > span')[i].text,
                    'price': dom.select('tr > td:nth-of-type(2) > span')[i].text
                    }) 
                
                for value in result:
                    producer.send(topicName, str(i).encode())
                    tim = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    producer.send(topicName, tim.encode())
                    print(value," - ")


NameError: name 'KafkaProducer' is not defined

In [ ]:
# 1월 7일 -> 하루가 지나고 갱신 
#=> 이중 딕셔너리 자체를 초기화
# 스케쥴러로 9시 시작하니까  -> 그 자체에서 초기화 하던가
########## 메인함수에 초반에 초기화 시키는 부분   ==> #토픽 초기화하는 함수 부분 추가하고
# 이어서 append..!

#1 => 9:00 - 9:04
#2 => 9:05 - 9:09
#3 => 9:10 - 9:14

# Jan 09 수정

## def stock => 형태 완성

In [18]:
#예진언니 코드 인용 ver

codes = {'336260', '009830', '086520', '011930', '018000', '035720', '035420', '036570', 
'041140', '018260', '005930', '000660', '011790', '014680', '000990', '023530', '004170', 
'008770', '028260', '026960', '105560', '071050', '055550', '316140', '086790', '047080', 
'089890', '030350', '230980', '069410'}



#starttime => 함수(stock)밖에서 9:05, 9:10 ~
result = {}
#starttime => 9:00 ~ 16:00 사이인지 체크하는 부분

def stock(result, starttime):
    now = datetime.datetime.now()
    date = datetime.datetime(now.year, now.month, now.day)
    
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
    codestock = defaultdict(lambda : list()) #result (dict)의 value 안에 들어갈 dict의 이름 : codestock
    # ticker 처음부터 다 돌기
    if starttime < 87:
        for code in codes:


            url = 'https://finance.naver.com/item/sise_time.nhn?code={}&thistime={}&page={}'.format(code, date.strftime("%Y%m%d")+"161122", 1)
            resp = download(url, headers = headers)
            dom = BeautifulSoup(resp.content, 'lxml')

            price = dom.select('tr > td:nth-of-type(2) > span')
            time = dom.select('tr > td:nth-of-type(1) > span')


                # result = for t, p in zip([i.text for i in dom.select('tr > td:nth-of-type(1) > span')], [i.text for i in dom.select('tr > td:nth-of-type(2) > span')])}
            for i in range(len(price)): # 한 종목의 주가
                    # result list에 dictionary 형태로 추가
                #if i < 5: #5개라고 생각했던 것 => 숫자가 랜덤하게 나오니 수정필요
                    
                    current_time = time[i].text #time[i].text가 원하는 시간대 안에 있는 건지
                    #time = datetime.datetime.strptime(time, "%H:%M")
                    #if문 추가해서 확인 time[i].text가 원하는 시간대 안에 있는 건지
                    if current_time 
                        current_price = price[i].text 
                        codestock[code].append((date.strftime("%Y.%m.%d") + " " + current_time, current_price) )
                    else:
                        break

        result[starttime] = codestock
        starttime += 1
        
    else:
        break
        
    return result
        

In [13]:
stock(result)

{1: defaultdict(<function __main__.stock.<locals>.<lambda>()>,
             {'105560': [('2022.01.10 15:58', '57,800'),
               ('2022.01.10 15:56', '57,800'),
               ('2022.01.10 15:54', '57,800'),
               ('2022.01.10 15:53', '57,800'),
               ('2022.01.10 15:52', '57,800')],
              '069410': [('2022.01.10 15:54', '10,350'),
               ('2022.01.10 15:51', '10,350'),
               ('2022.01.10 15:40', '10,350'),
               ('2022.01.10 15:30', '10,350'),
               ('2022.01.10 15:19', '10,400')],
              '036570': [('2022.01.10 15:57', '614,000'),
               ('2022.01.10 15:53', '614,000'),
               ('2022.01.10 15:52', '614,000'),
               ('2022.01.10 15:49', '614,000'),
               ('2022.01.10 15:48', '614,000')],
              '086520': [('2022.01.10 15:58', '98,300'),
               ('2022.01.10 15:56', '98,300'),
               ('2022.01.10 15:55', '98,300'),
               ('2022.01.10 15:54', '98,300

# 5분마다 스케쥴링

In [14]:
schedule.every(5).minutes.do(stock)


Every 5 minutes do stock() (last run: [never], next run: 2022-01-10 20:59:03)

In [ ]:
# 1) 0분 / 5분 / 10분 / 15분 
# 2) 

In [19]:
import time
import random
import datetime
from kafka import KafkaProducer

In [28]:
def message():
    start = datetime.datetime.now()
    print("스케쥴 실행중...")
    for key,value in result.items():
        realtimestock = stock(result, 1)
        
    end = datetime.datetime.now()
    print("실행 : ", end - start)

In [29]:
bootstrap_servers = ['localhost:9092']
topicName = 'realtime_stock'
producer = KafkaProducer(bootstrap_servers = bootstrap_servers)

job = schedule.every(60).seconds.do(message)

while True:
    
    NOW = datetime.datetime.now()
    schedule.run_pending()
    if datetime.datetime.now() == NOW + datetime.timedelta(minutes = 1):
        producer.send(topicName, str(job()).encode()) ##
        tim = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')    
        producer.send(topicName, tim.encode())
        break

TypeError: stock() missing 1 required positional argument: 'result'